In [1]:
import robotic as ry
import time

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('Question-01/environment/cargo.g'))
C.addFile(ry.raiPath('Question-01/environment/cargobot.g'))
C.addFile(ry.raiPath('Question-01/environment/maze.g'))

C.view()

0

In [74]:
# del C
# C.view()
# C.getFrameNames()
# qHome = C.getJointState()
# print(qHome)
# print(C.getFrameNames())

### 1.1 Part A: gripper to cargo handle position KOMO_1

In [ ]:
komo = ry.KOMO(C, phases=1, slicesPerPhase=30, kOrder=1, enableCollisions=False)

# add control objective to stay close to the initial joint configuration
komo.addControlObjective([], 0, 1e-1)

# add collision constraints
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, 1e1)

# move the gripper to the cargo position
komo.addObjective(
    times=[],
    feature=ry.FS.positionDiff,
    frames=["l_gripper", "cargo_handle"],
    type=ry.OT.sos,
    scale=[1e1]
)

# align the grippper with the cargo_handle
komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', 'cargo_handle'], ry.OT.eq, [1e0], [0])
# komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper', 'cargo_handle'], ry.OT.eq, [1e0], [0])



ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q_gripper_to_cargo = komo.getPath()

for t in range(q_gripper_to_cargo.shape[0]):
    C.setJointState(q_gripper_to_cargo[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

C.view()

{ time: 0.027499, evals: 11, done: 1, feasible: 1, sos: 2.84254, f: 0, ineq: 0, eq: 0.0197295 }


0

In [ ]:
# C.attach("l_gripper", "cargo_handle")
# del komo
# C.getFrame("base_origin").setPosition([1.7, -1.6, 0.1])
# C.getFrame("base_origin").setQuaternion([1, 0, 0, 0])
# C.view()

0

### 1.1 Part A: l_panda_base to goal position KOMO_2

In [3]:
komo = ry.KOMO()
komo = ry.KOMO(C, phases=1, slicesPerPhase=30, kOrder=1, enableCollisions=False)

# add control objective to stay close to the initial joint configuration
komo.addControlObjective([], 0, 1e-1)

# add collision constraints
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

# Add objective to move cargo to goal area
komo.addObjective(
    times=[1., 2.],
    feature=ry.FS.positionDiff,
    frames=["l_panda_base", "goal_area"],
    type=ry.OT.sos
)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q_cargobot_to_goal_area = komo.getPath()

for t in range(q_cargobot_to_goal_area.shape[0]):
    C.setJointState(q_cargobot_to_goal_area[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.008145, evals: 26, done: 1, feasible: 1, sos: 0.299811, f: 0, ineq: 0, eq: 0 }


In [82]:
for t in range(q_gripper_to_cargo.shape[0]):
    C.setJointState(q_gripper_to_cargo[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)
    
for t in range(q_cargobot_to_goal_area.shape[0]):
    C.setJointState(q_cargobot_to_goal_area[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

### 1.2 Part B: RRT pathfinding